This is the Part-8 of the Deep Reinforcement Learning Notebook series. In this Notebook I have introduced Proximal Policy Optimization (PPO). 

The Notebook series is about Deep RL algorithms so it excludes all other techniques that can be used to learn functions in reinforcement learning and also the Notebook Series is not exhaustive i.e. it contains the most widely used Deep RL algorithms only.

# What is Proximal Policy Optimization (PPO)
Proximal Policy Optimization (PPO) was intoduced by Schulman et al in the paper "Proximal Policy Optimization Algorithms” (2017).It is a another family of policy gradient methods for reinforcement learning, which alternate between sampling data through interaction with the environment, and optimizing a "surrogate" objective function using stochastic gradient ascent.

# Challenge when training agents with older policy gradient algorithms 
One challenge when training agents with policy gradient algorithms is that they are susceptible to performance collapse, in which an agent suddenly starts to perform badly. This scenario can be hard to recover from because an agent will start to generate poor trajectories, which are then used to further train the policy. Second is that on-policy algorithms are sample-inefficient because they cannot reuse data.PPO is class of optimization algorithms that addresses these two issues.

The main idea behind PPO is to introduce a surrogate objective which avoids performance collapse by guaranteeing monotonic policy improvement. This objective also has the benefit of permitting off-policy data to be reused.
PPO can be used to extend REINFORCE or Actor-Critic by replacing the original objective J($π_θ$) with the modified PPO objective. This modification leads to more stable and more sample-efficient training.

#Performance Collapse
When we discussed policy gradient algorithms we learned about Policy Objective Function.We know that a policy $π_θ$ is optimized by changing the policy parameters θ using the policy gradient $▽_θJ(π_θ)$.This is an indirect approach because we are searching for the optimal policy in the policy space by making changes in parameters in parameter space. Unfortunately the policy space and the parameter space do not always map congruently, and the distances in both spaces don’t necessarily correspond.Say $θ_1, θ_2$ and $θ_2, θ_3$ and suppose they have the same distance in the parameter space, that is, $d_θ(θ_1, θ_2) = d_θ(θ_2, θ_3)$. However, their mapped policies $π_{θ_1}, π_{θ_2}$ and $π_{θ_2}, π_{θ_3}$ do not neccessarily have the same distance.
This presents a potential problem because the ideal step size α for a parameter update becomes difficult to determine. It is not clear beforehand how small or large a step in the policy space Π it is going to map to. If α is too small, it will require many iterations and training will take a long time. Alternatively, the policy may also get stuck in some local maxima. If α is too large, the corresponding step in policy space will overshoot the neighborhood of good policies and cause a performance collapse. When this happens, the new policy that is much worse will be used to generate bad trajectory data for the next update and ruin future policy iterations. Because of this, a performance collapse can be very difficult to recover from. In general, a constant step size does not avoid these issues. 



The optimal choice for α will vary depending on where the current policy $π_θ$ is in the policy space Π, and how the current parameter θ maps to the neighborhood of $π_θ$ in Π from its own neighborhood in Θ. Ideally an algorithm should adaptively vary the parameter step size based on these factors.
To derive an adaptive step size based on how a particular update in parameter space will affect policy space, we first need way to measure the difference in performance between two policies.


# Relative Policy Performance Identity And Surrogate Objective
We can define a relative policy performance identity as the difference in the objectives between them, shown in Equation 7.1.

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%204.29.31%20PM.png?raw=true)

Equation 7.1

The relative policy performance identity J(π′) – J(π) serves as a metric used to measure policy improvements. If the difference is positive, the newer policy π′ is better than π. During policy iteration, we should ideally choose a new policy π′ such that this difference is maximized. Therefore, maximizing objective J(π′) is equivalent to maximizing this identity, and they can both be done by gradient ascent.

Framing the objective this way also means that every policy iteration should ensure a nonnegative (monotonic) improvement, i.e. J(π′) – J(π)≥ 0, since in the worst case we can simply let π′ = π to obtain no improvement. With this, there will be no performance collapses throughout the training, which is the property we are looking for.

However, there is a limitation that prohibits the identity from being used as an objective function. Note that in its expression $E_{τ∼π′}[ ∑_{t≥0} A^π(s_t, a_t)]$, the expectation requires trajectories to be sampled from the new policy π′ for an update, but π′ is not available until after the update. To remedy this paradox, we need to find a way to alter it such that it uses the old policy π that is available.
To this end, we can assume that successive policies π, π′ are relatively close (this can be measured by low KL divergence), such that the state distributions from both are similar.

Then, Equation 7.1 can be approximated by using trajectories from the old policy, τ ∼ π, adjusted with importance sampling weights $π′(a_t/s_t)$/$π(a_t/s_t)$. This adjusts the returns generated using π by the ratio of action probabilities between the two successive policies π, π . The rewards associated with actions that are more likely under the new policy π′ will be up-weighted, and rewards associated with relatively less actions under π′ will be down- weighted. This approximation is shown in Equation 7.2.
![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%204.42.26%20PM.png?raw=true)

Equation 7.2

The new objective on the right hand side of Equation 7.2 $J_π^{CPI}(π')$ , is called a surrogate objective because it contains
a ratio of the new and old policies, π′ and π. The superscript CPI stands for “conservative policy iteration”.

The proof is skipped but it can be show that the gradient of the surrogate objective is equal to the policy gradient, as stated in Equation 7.3.

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%204.49.10%20PM.png?raw=true)

Equation 7.3

Before $J_π^{CPI}(π')$ can stand in as the new objective for a policy gradient algorithm, there is one final requirement to satisfy.
$J_π^{CPI}(π')$ is only an approximation for J(π′) – J(π), so it will contain some error. However, we want to guarantee when we
use $J_π^{CPI}(π')$ ~ J(π′) – J(π) that J(π′) – J(π) ≥ 0. We therefore need to understand the error introduced by the
approximation.We express the absolute error by taking the difference between the new objective J(π′) and its estimated improvement J(π)+ $J_π^{CPI}(π')$.Then, the error can be bounded in terms of the KL divergence between π and π.

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%205.00.16%20PM.png?raw=true)

Equation 7.3

C is a constant that needs to be chosen, and KL is the KL divergence.

Using this, it is quite straightforward to derive the result we are after, namely 

J(π′) – J(π) ≥ 0. This is called the monotonic improvement theory.

If we add the error bound as a penalty in our optimization problem, then we can guarantee monotonic policy improvement. Our optimization problem now becomes the following.

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%205.06.17%20PM.png?raw=true)

Equation 7.4

This result satisfies our final requirement. It allows us to avoid performance collapse that may occur when using the original objective J(π). One key distinction to note is that a monotonic improvement does not guarantee convergence to an optimal policy π*. For instance, policy optimization can still get stuck at a local maxima in which every policy iteration produces no improvement, i.e. J(π′) – J(π) = 0. Guaranteeing convergence remains a difficult open problem.
The final step step is to consider how to implement the optimization problem from Equation 7.4 in practice. One idea is to directly constrain the expectation of KL as shown in Equation 7.5.

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%205.08.45%20PM.png?raw=true)

Equation 7.5

δ limits how large KL divergence can be, so it effectively restricts how far a new policy π′ can diverge away from the old policy π. Only candidates in a close neighborhood around π in the policy space will be considered. This neighborhood is called a trust region, and Equation 7.5 is called a trust region constraint. Note that δ is a hyperparameter that needs to be tuned.


Putting the constraint and surrogate objective together, the trust region policy optimization problem is shown in Equation 7.6.
![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%206.16.07%20PM.png?raw=true)

Equation 7.6

To sum up, $J_π^{CPI}(π')$ is a linear approximation to J(π′) – J(π)
since its gradient is equal to the policy gradient. It also guarantees monotonic improvement to within an error bound. To ensure improvement accounting for this potential error, we impose a trust region constraint to limit the difference between the new and the old policies. Provided changes in the policy stay within the trust region, we can avoid performance collapse.

# PROXIMAL POLICY OPTIMIZATION (PPO)

PPO is easy to implement, computationally inexpensive, and we do not have to choose δ. For these reasons, it has become one of the most popular policy gradient algorithms.
PPO is a family of algorithms that solves the trust-region constrained policy optimization problem with simpler and effective heuristics. Two variants exist. The first is based on an adaptive KL penalty, and the second is based on a clipped objective.

The first variant of PPO is called PPO with adaptive KL penalty, with the objective shown in Equation 7.7. It turns the KL constraint $E_t[KL(π_θ(a_t|s_t)||π_{old}(a_t|s_t))] ≥ δ $ into an adaptive KL penalty which is subtracted from the importance-weighted advantage. The expectation of the result is the new objective to be maximized.

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%206.21.21%20PM.png?raw=true)


Equation 7.7

Equation 7.7 is known as a KL-penalized surrogate objective. β is an adaptive coefficient which controls the size of the KL penalty. It serves the same purpose as Equation 7.32 in controlling the trust region of optimization. The larger β the more the difference between $π_θ$ and $π_{θold}$. The smaller β the higher the tolerance between the two policies.

One challenge of using a constant coefficient is that different problems have different characteristics, so it is hard to find one which will work for all of them. Even on a single problem, the loss landscape changes as policy iterates, so a β that works earlier may not work later. β needs to adapt accordingly to these changes.
To tackle this problem, the PPO paper proposes a heuristic- based update rule for β that allows it to adapt over time. β is updated after every policy update and the new value is used in the next iteration. The update rule for β is shown below. It can be used as a subroutine in the PPO algorithm which will be presented later.

![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%206.27.24%20PM.png?raw=true)


At the end of each iteration, we estimate δ and compare it to the desired target $δ_{tar}$. If δ is less than $δ_{tar}$ by some margin, we decrease the KL penalty by reducing β. However, if δ is greater than $δ_{tar}$ by some margin, we increase the KL penalty by increasing β. The specific values used to determine the margins and the update rules for β are chosen empirically. The authors selected 1.5 and 2 respectively, but also found that the algorithm is not very sensitive to these values.
It was also observed that KL divergence occasionally departs far away from from its target value $δ_{tar}$, but β adjusts quickly. Good values for $δ_{tar}$ also needs to be determined empirically.
This approach has the benefit of being simple to implement. However, it hasn’t overcome the problem of needing to choose a target δ. Furthermore, it can be computationally expensive because of the need to calculate the KL.

PPO with clipped surrogate objective remedies this by doing away with the KL constraint and opts for a simpler modification to the surrogate objective as shown in Equation 7.8.


![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%206.30.42%20PM.png?raw=true)

Equation 7.8

Equation 7.8 is known as clipped surrogate objective. ∊ is a
value which defines the clipping neighborhood |$r_t$(θ) – 1| ≤ ∊. It
is a hyperparameter to be tuned, and can be decayed during
training. The first term within min(·) is simply the surrogate $J^{CPI}$.The second term clip ($r_t$(θ), 1 – ∊, 1 + ∊)At bounds the value of $J^{CPI}$ to be between (1 – ∊)$A_t$ and (1 + ∊)$A_t$. When $r_t$(θ) is within the interval [1 – ∊, 1 + ∊], both terms inside min(·) are equal. This objective prevents parameter updates which could cause large and risky changes to the policy πθ. For the purpose of quantifying large policy changes, the probability ratio rt(θ) is useful. 

The clipped objective $J^{CLIP}$ is cheap to compute, very easy to understand, and implementation requires only a few trivial modifications to the original surrogate objective $J^{CPI}$.
The most costly computations are the probability ratio rt(θ) and advantage At. However, these are the minimum calculations needed by any algorithms optimizing the surrogate objective. The other calculations are clipping and minimizing, which are essentially constant-time.

# PPO with clipping, extending Actor- Critic Algorithm
![alt text](https://github.com/Machine-Learning-rc/Unimportant/blob/master/Screenshot%202020-07-31%20at%206.46.01%20PM.png?raw=true)

# IMPLEMENTING PPO with clipping, extending Actor- Critic

Below code setups the environment required to run and record the game and also loads the required library.

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Sequential,load_model,Model
import gym
import numpy as np
import random
from collections import deque
from tensorflow.keras.utils import normalize as normal_values
import cv2
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython.display import clear_output
from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

This part ensures the reproducibility of the code below by using a random seed and setups the environment.

In [ ]:
RANDOM_SEED=3
# random seed (reproduciblity)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# set the env
env = (gym.make("Ant-v2"))  # env to import
#env=wrap_env(env)  #use this when you want to record a video of episodes

env.seed(RANDOM_SEED)
env.reset() # reset to env
upper_bound = env.action_space.high[0]
lower_bound = env.action_space.low[0]

Defining the PPO Class. At initiation, the PPO object sets a few parameters like environment, action and state space,create Actor,Critic,and target Actor models,and remember(that records the observations of each step.)

In [ ]:
class PPO:
 
  def __init__(self,Actor_path=None,Critic_path=None,old_Actor_path=None):
    #self.env=env #import env
    self.state_shape=111 # the state space
    self.action_shape=env.action_space.shape[0] # the action space
    self.upper_bound=upper_bound
    self.lower_bound=lower_bound
    self.gamma=[0.99] # decay rate of past observations
    self.learning_rate=3e-5 # learning rate in deep learning
    self.lambda_=0.95
    self.epochs=7
    self.tau=0.005
    self.batch_size=64
    self.beta=0.001 #Entropy Loss ratio
    self.clipping_ratio = 0.2
    if not Actor_path:
      self.Actor_model=self._create_model('Actor')    #Target Model is model used to calculate target values
      self.target_Actor_model=self._create_model('Actor')
      self.Critic_model=self._create_model('Critic')  #Training Model is model to predict q-values to be used.
    else:
      self.Actor_model=load_model(Actor_path) #import model
      self.Critic_model=load_model(Critic_path) #import model
      self.target_Actor_model=load_model(old_Actor_path)
    
        # record observations
    self.states=deque()
    self.rewards=deque()
    self.actions=deque()
    self.last_state=np.zeros((1,self.state_shape))
  
  def remember(self, state,reward,action,last_state,done):
    '''stores observations'''
    self.states.append(state)
    self.actions.append(action)
    self.rewards.append(reward)
    if done:
      self.last_state[0]=last_state

Creating a Neural Network Model (Actor and Critic)

In [ ]:
def _create_model(self,model_type):
 
    ''' builds the model using keras'''
 
    state_input=Input(shape=(self.state_shape,))
    layer_1=Dense(128,activation='relu')(state_input)
    layer_2=Dense(128,activation='relu')(layer_1)
    
 
    if model_type=='Actor':
      output =(Dense(2*self.action_shape, activation=None))(layer_2)
      model = Model(inputs=[state_input],outputs=[output])
    else:
      output=Dense(1, activation=None,kernel_initializer=tf.keras.initializers.random_uniform(minval=-0.0003,maxval=0.0003))(layer_2)
      model = Model(inputs=[state_input],outputs=[output])
      model.compile(optimizer=Adam(learning_rate=self.learning_rate),loss="mse")
    return model

Action Selection  and Probability Calcualtion

The get_action method guides the action choice. We get mean and standard deviation from Actor Model and then we use them to get action. Then we clip the action so that action doesn't got out of legal actions range.


The log_policy_prob method provides us with the probability of the action This method is used for caculating probs later while training the model.


In [ ]:
  def get_action(self, state,status="Training"):
    action_mean,action_std=self.Actor_model(state)
    action = action_mean.numpy() + np.exp(action_std.numpy()) * np.random.normal(size=action_std.shape)
    action=action*self.upper_bound
    legal_action = np.clip(action, self.lower_bound, self.upper_bound)
    return legal_action[0]

  def log_policy_prob(self,mean, std, action):
    # policy log probability
    exp_coeff= -(((action-mean)/(np.exp(std)))**2)/2
    act_log_softmax = tf.math.exp(exp_coeff)/(tf.math.sqrt(2*math.pi)*(np.exp(std)))
    return tf.reduce_sum(act_log_softmax,axis=1,keepdims=True)


The get_GAEs method calculates Generalized advantage estimation (GAE) which later is used to calculate Advantage.

In [ ]:
  def get_GAEs(self,v_preds,rewards):
    T = len(v_preds)-1
    gaes = np.zeros((T,1),dtype='float32')
    future_gae = 0.0
    for t in reversed(range(T)):
      delta = rewards[t] + np.asarray(self.gamma) * v_preds[t + 1] - v_preds[t]
      gaes[t] = future_gae = delta + np.asarray(self.gamma) * np.asarray(self.lambda_) *np.asarray(future_gae)
    return gaes

Updating networks

We update critic and actor according to loss that we discussed earlier and do soft updates on target network.

In [ ]:
  def update_models(self):

    states_mb=np.zeros((self.batch_size,self.state_shape))
    V_s_mb=np.zeros((self.batch_size,1))
    actions_mb=np.zeros((self.batch_size,self.action_shape))
    rewards_mb=np.zeros((self.batch_size,1))

    batch_indices = np.random.choice(len(self.states),self.batch_size)

    for i,j in enumerate(batch_indices):
      
      state_=self.states[j]
      states_mb[i]=state_
      actions_mb[i]=(self.actions[j])
      rewards_mb[i]=self.rewards[j]

      
      vs_=self.Critic_model.predict(state_)
      V_s_mb[i]=vs_

    
    
    
    V_last_state=self.Critic_model.predict(self.last_state)
    v_all=np.concatenate((V_s_mb,V_last_state),axis=0)
    
    Advantages=self.get_GAEs(v_all,rewards_mb)
    critic_targets = Advantages +  V_s_mb
    self.Critic_model.fit(states_mb, critic_targets,epochs=self.epochs,verbose=2)
    
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)

    def train_step(states_1_mb,Advantages,actions):
      with tf.GradientTape() as tape:
        Advantages=tf.stop_gradient(Advantages)
        mean_and_log_std=self.Actor_model(states_1_mb,training=True)
        mean,std = tf.split(mean_and_log_std, num_or_size_splits=2, axis=1)
        probs=self.log_policy_prob(mean,std,actions)
        
        old_mean_and_log_std = (self.old_Actor_model)(states_1_mb,training=True)
        old_mean,old_std = tf.split(old_mean_and_log_std, num_or_size_splits=2, axis=1)
        old_probs=self.log_policy_prob(old_mean,old_std,actions)
        old_probs=tf.stop_gradient(old_probs)
        
        ratio =tf.math.exp(probs-old_probs)
        clip_ratio = tf.clip_by_value(ratio, clip_value_min=1 - self.clipping_ratio, clip_value_max=1 + self.clipping_ratio)
        surrogate=(tf.math.minimum(ratio*Advantages,clip_ratio*Advantages))
        entropy_loss=-(probs * tf.math.log(probs+1e-10))
        ppo_loss = -(tf.math.reduce_mean(surrogate+self.beta* entropy_loss))                      
      grads = tape.gradient(ppo_loss,self.Actor_model.trainable_variables)
      grads = [(tf.clip_by_value(grad, -1.0, 1.0)) for grad in grads]
      optimizer.apply_gradients(zip(grads, self.Actor_model.trainable_variables))
      return ppo_loss

    ppo_loss=[]
    for epoch in range(self.epochs):
      loss=train_step(states_mb,Advantages,actions_mb)
      ppo_loss.append(loss)
    print("PPO_loss:",np.mean(ppo_loss)) 
   
    actor_weights = np.array(self.Actor_model.get_weights())
    actor_target_weights = np.array(self.old_Actor_model.get_weights())
    new_weights = self.tau*actor_weights + (1-self.tau)*actor_target_weights
    self.old_Actor_model.set_weights(new_weights)

    self.states.clear();self.rewards.clear();self.actions.clear()

Training the model

This method creates a training environment for the model. Iterating through a set number of episodes, it uses the model to sample actions and play them. When such a sequence ends, the model is using the recorded observations to update the policy.

In [ ]:
  def train(self,episodes):
    env = (gym.make("Ant-v2"))
    done_1=False
    for episode in range(episodes):
      T_max=1000
      state=env.reset()
      done=False
      state_=state.reshape((1,111))
      episode_reward=0  
      print("Episode Started")
      for t in range(T_max):
        action=self.get_action(state_)
        next_state, reward, done, info=env.step(action)
        reward_=reward
        next_state_=next_state.reshape((1,111))
        if t==T_max-1 or done:
          done_1=True
        self.remember(state_,reward,action,next_state_,done_1)
        state_ = next_state_
        episode_reward+=reward
        if done:
          break
      self.Average_rewards.append(episode_reward)
      avg_reward = np.mean(self.Average_rewards[-100:])
      print('Episode Ended')
      print("Episode:{}  Reward:{} Average_reward:{}".format(episode,episode_reward,avg_reward))
      print("Updating the model")
      self.update_models()
      done_1=False
      if episode%100==0 and episode!=0:
        self.Actor_model.save('Ant-v2_Actor_{}.h5'.format(episode))
        self.old_Actor_model.save('Ant-v2_old_Actor_{}.h5'.format(episode))
        self.Critic_model.save('Ant-v2_Critic_{}.h5'.format(episode))

In [ ]:
Agent_2=PPO()
Agent_2.train(10000)